In [68]:
import pandas as pd

df = pd.read_csv('cleaned_diamond_data.csv').rename(columns = {'Unnamed: 0' : 'id'})
df.drop('id',inplace=True,axis=1)
df.head(5)

,carat,clarity,color,cut,x,y,z,depth,table,price
0,0.5,IF,D,IDEAL,5.1,5.15,3.20,61.5,56.0,3000.0
1,0.7,VVS2,E,PREMIUM,5.7,5.49,3.52,62.0,59.0,4500.0
2,0.5,SI2,H,GOOD,4.3,4.31,3.90,62.3,56.0,700.0
3,1.2,IF,D,IDEAL,5.9,6.82,4.20,61.7,58.0,10000.0
4,0.9,I1,J,FAIR,6.0,5.49,3.70,61.7,56.0,2400.0


In [69]:
# Przekształć dane obiektowe na liczby
le = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

df['clarity'] = le.fit_transform(df['clarity'])
df['cut'] = le2.fit_transform(df['cut'])
df['color'] = le3.fit_transform(df['color'])

In [110]:
model = smf.ols(formula="price ~ carat+ clarity + color +  cut + table + x+ y+z ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     2.635
Date:                Tue, 30 Jan 2024   Prob (F-statistic):            0.00924
Time:                        16:44:37   Log-Likelihood:                -2390.6
No. Observations:                 200   AIC:                             4799.
Df Residuals:                     191   BIC:                             4829.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.779e+05   1.27e+05     -1.399      0.163   -4.29e+05    7.29e+04
carat      -1.164e+04   1.44e+04     -0.808      0.420   -4.01e+04    1.68e+04
clarity    -1317.2409   1671.040     -0.788      0.432   -4613.303    1978.821
color       -856.1343   1538.285     -0.557      0.578   -3890.342    2178.074
cut          550.2560   2032.052      0.271      0.787   -3457.889    4558.401
table       3188.8084   2210.532      1.443      0.151   -1171.382    7548.999
x           2.595e+04   9195.698      2.822      0.005    7815.856    4.41e+04
y          -2.029e+04   6643.716     -3.054      0.003   -3.34e+04   -7182.628
z          -3873.2520   1.08e+04     -0.358      0.721   -2.52e+04    1.74e+04
==============================================================================
Omnibus:                      265.246   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12930.672
Skew:                           5.859   Prob(JB):                         0.00
Kurtosis:                      40.608   Cond. No.                     2.70e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.7e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [111]:
model = smf.ols(formula="price ~ carat+ clarity + color +  cut + table + x+z ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     1.610
Date:                Tue, 30 Jan 2024   Prob (F-statistic):              0.135
Time:                        16:44:45   Log-Likelihood:                -2395.3
No. Observations:                 200   AIC:                             4807.
Df Residuals:                     192   BIC:                             4833.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.506e+05   1.28e+05     -1.964      0.051   -5.02e+05    1059.036
carat      -6146.2674   1.46e+04     -0.421      0.674    -3.5e+04    2.27e+04
clarity    -1272.8962   1706.816     -0.746      0.457   -4639.413    2093.621
color      -1165.9537   1567.856     -0.744      0.458   -4258.388    1926.480
cut          812.6705   2073.778      0.392      0.696   -3277.642    4902.983
table       3813.2571   2248.261      1.696      0.091    -621.205    8247.719
x           8276.4092   7297.932      1.134      0.258   -6118.007    2.27e+04
z           1947.5620   1.09e+04      0.179      0.858   -1.95e+04    2.34e+04
==============================================================================
Omnibus:                      280.986   Durbin-Watson:                   2.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16637.324
Skew:                           6.400   Prob(JB):                         0.00
Kurtosis:                      45.810   Cond. No.                     2.64e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.64e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [112]:
model = smf.ols(formula="price ~  clarity + color +  cut +table + x+z ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     1.856
Date:                Tue, 30 Jan 2024   Prob (F-statistic):             0.0903
Time:                        16:45:12   Log-Likelihood:                -2395.4
No. Observations:                 200   AIC:                             4805.
Df Residuals:                     193   BIC:                             4828.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2.391e+05   1.24e+05     -1.923      0.056   -4.84e+05    6186.562
clarity    -1235.5608   1700.870     -0.726      0.468   -4590.241    2119.120
color      -1212.8291   1560.556     -0.777      0.438   -4290.764    1865.105
cut          820.4622   2069.270      0.396      0.692   -3260.824    4901.748
table       3739.2713   2236.592      1.672      0.096    -672.030    8150.573
x           6481.7262   5909.438      1.097      0.274   -5173.646    1.81e+04
z           1645.0193   1.08e+04      0.152      0.879   -1.97e+04     2.3e+04
==============================================================================
Omnibus:                      281.266   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16676.549
Skew:                           6.411   Prob(JB):                         0.00
Kurtosis:                      45.858   Cond. No.                     2.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [113]:
model = smf.ols(formula="price ~ carat+ color +  cut +x +z ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.613
Date:                Tue, 30 Jan 2024   Prob (F-statistic):              0.158
Time:                        16:45:39   Log-Likelihood:                -2397.0
No. Observations:                 200   AIC:                             4806.
Df Residuals:                     194   BIC:                             4826.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.264e+04   3.29e+04     -1.296      0.197   -1.08e+05    2.23e+04
carat      -3885.2397   1.46e+04     -0.266      0.790   -3.27e+04    2.49e+04
color      -1401.4541   1548.479     -0.905      0.367   -4455.469    1652.560
cut         1827.0443   1996.860      0.915      0.361   -2111.297    5765.386
x           9154.2175   7302.430      1.254      0.212   -5248.128    2.36e+04
z           1175.7817   1.08e+04      0.109      0.914   -2.02e+04    2.25e+04
==============================================================================
Omnibus:                      284.733   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17512.852
Skew:                           6.539   Prob(JB):                         0.00
Kurtosis:                      46.938   Cond. No.                         103.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [114]:
model = smf.ols(formula="price ~  clarity +color+   cut  + x+z ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.653
Date:                Tue, 30 Jan 2024   Prob (F-statistic):              0.148
Time:                        16:46:03   Log-Likelihood:                -2396.9
No. Observations:                 200   AIC:                             4806.
Df Residuals:                     194   BIC:                             4826.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.638e+04   2.78e+04     -1.311      0.192   -9.11e+04    1.84e+04
clarity     -871.1900   1694.636     -0.514      0.608   -4213.465    2471.085
color      -1528.6960   1556.228     -0.982      0.327   -4597.994    1540.602
cut         1679.7801   2013.671      0.834      0.405   -2291.717    5651.277
x           7866.6279   5878.100      1.338      0.182   -3726.559    1.95e+04
z           1608.8370   1.09e+04      0.148      0.883   -1.98e+04    2.31e+04
==============================================================================
Omnibus:                      284.890   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17617.172
Skew:                           6.542   Prob(JB):                         0.00
Kurtosis:                      47.078   Cond. No.                         88.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [115]:
model = smf.ols(formula="price ~ carat+ clarity +   cut +x +z ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.478
Date:                Tue, 30 Jan 2024   Prob (F-statistic):              0.199
Time:                        16:46:18   Log-Likelihood:                -2397.3
No. Observations:                 200   AIC:                             4807.
Df Residuals:                     194   BIC:                             4826.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -5.188e+04   3.12e+04     -1.661      0.098   -1.13e+05    9712.339
carat      -5078.4579   1.46e+04     -0.348      0.728   -3.39e+04    2.37e+04
clarity     -693.8227   1687.283     -0.411      0.681   -4021.595    2633.950
cut         1969.1075   1996.536      0.986      0.325   -1968.595    5906.810
x            1.01e+04   7242.644      1.395      0.165   -4184.001    2.44e+04
z           1414.1126   1.09e+04      0.130      0.897   -2.01e+04    2.29e+04
==============================================================================
Omnibus:                      286.950   Durbin-Watson:                   2.061
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18308.450
Skew:                           6.613   Prob(JB):                         0.00
Kurtosis:                      47.968   Cond. No.                         93.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [116]:
model = smf.ols(formula="price ~  clarity +   cut  +x+z ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.825
Date:                Tue, 30 Jan 2024   Prob (F-statistic):              0.125
Time:                        16:46:43   Log-Likelihood:                -2397.4
No. Observations:                 200   AIC:                             4805.
Df Residuals:                     195   BIC:                             4821.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -4.59e+04    2.6e+04     -1.765      0.079   -9.72e+04    5396.376
clarity     -664.3896   1681.357     -0.395      0.693   -3980.368    2651.589
cut         1967.8217   1992.027      0.988      0.324   -1960.862    5896.506
x           8611.2519   5828.494      1.477      0.141   -2883.727    2.01e+04
z           1154.7522   1.09e+04      0.106      0.915   -2.03e+04    2.26e+04
==============================================================================
Omnibus:                      287.232   Durbin-Watson:                   2.064
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18367.345
Skew:                           6.624   Prob(JB):                         0.00
Kurtosis:                      48.040   Cond. No.                         75.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [125]:
model = smf.ols(formula="price ~     cut +x    ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     3.604
Date:                Tue, 30 Jan 2024   Prob (F-statistic):             0.0290
Time:                        16:48:49   Log-Likelihood:                -2397.4
No. Observations:                 200   AIC:                             4801.
Df Residuals:                     197   BIC:                             4811.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.568e+04   2.19e+04     -2.085      0.038   -8.89e+04   -2477.005
cut         2059.2938   1968.815      1.046      0.297   -1823.364    5941.952
x           8972.9535   3633.497      2.470      0.014    1807.410    1.61e+04
==============================================================================
Omnibus:                      287.134   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18277.154
Skew:                           6.623   Prob(JB):                         0.00
Kurtosis:                      47.920   Cond. No.                         50.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\patsy\compat.py:36, in call_and_wrap_exc(
    msg='Error evaluating factor',
    origin=EvalFactor('cut'),
    f=<bound method EvalEnvironment.eval of <patsy.eval.EvalEnvironment object>>,
    *args=('cut',),
    **kwargs={'inner_namespace': VarLookupDict([{},      carat    x     y  table
...2  5.49   57.0
76    1.30  6.9  6.92   59.0, {}])}
)
     35 try:
---> 36     return f(*args, **kwargs)
        f = <bound method EvalEnvironment.eval of <patsy.eval.EvalEnvironment object at 0x0000011E716E62F0>>
        args = ('cut',)
        kwargs = {'inner_namespace': VarLookupDict([{},      carat    x     y  table
95    0.50  6.4  6.42   56.0
15    0.50  4.4  4.39   57.0
30    0.65  5.6  5.49   58.0
158   0.50  6.4

In [100]:
df.corr()

,const,carat,clarity,color,cut,x,y,z,depth,table,price
const,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
carat,NaN,1.000000,0.038356,-0.083564,0.006939,0.780244,0.536506,0.632880,0.266465,0.223510,0.121116
clarity,NaN,0.038356,1.000000,-0.114816,-0.117444,0.084047,0.041925,0.131219,0.110232,0.124603,-0.020934
color,NaN,-0.083564,-0.114816,1.000000,-0.130213,-0.160668,-0.074220,-0.106440,0.071364,-0.195118,-0.100818
cut,NaN,0.006939,-0.117444,-0.130213,1.000000,0.003132,-0.045412,-0.003561,-0.025157,0.242646,0.073736
x,NaN,0.780244,0.084047,-0.160668,0.003132,1.000000,0.758170,0.779332,0.314368,0.238023,0.173042
y,NaN,0.536506,0.041925,-0.074220,-0.045412,0.758170,1.000000,0.516829,0.207827,0.101011,-0.012701
z,NaN,0.632880,0.131219,-0.106440,-0.003561,0.779332,0.516829,1.000000,0.314184,0.188741,0.137292
depth,NaN,0.266465,0.110232,0.071364,-0.025157,0.314368,0.207827,0.314184,1.000000,0.027876,0.050543
table,NaN,0.223510,0.124603,-0.195118,0.242646,0.238023,0.101011,0.188741,0.027876,1.000000,0.171113


In [95]:
import statsmodels.formula.api as smf

model = smf.ols(formula="price ~ carat ", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     2.948
Date:                Tue, 30 Jan 2024   Prob (F-statistic):             0.0876
Time:                        16:34:20   Log-Likelihood:                -2399.6
No. Observations:                 200   AIC:                             4803.
Df Residuals:                     198   BIC:                             4810.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -96.3184   7166.174     -0.013      0.989   -1.42e+04     1.4e+04
carat       1.564e+04   9112.356      1.717      0.088   -2324.826    3.36e+04
==============================================================================
Omnibus:                      291.398   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19630.243
Skew:                           6.776   Prob(JB):                         0.00
Kurtosis:                      49.604   Cond. No.                         5.08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [128]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash import dash_table
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import plotly.graph_objects as go


# Wczytanie danych
df = pd.read_csv('cleaned_diamond_data.csv').rename(columns={'Unnamed: 0': 'id'})
df.drop('id', inplace=True, axis=1)

# Label Encoding dla danych kategorycznych
le = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

df['clarity'] = le.fit_transform(df['clarity'])
df['cut'] = le2.fit_transform(df['cut'])
df['color'] = le3.fit_transform(df['color'])

# Podział na zbiór treningowy i testowy
y = df['price']
X = df.drop(columns=['price'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Budowa modelu regresji
model = LinearRegression()

# Eliminacja wsteczna - RFE (Recursive Feature Elimination)
selector = RFE(model, step=1)
selector = selector.fit(X_train, y_train)

# Wybrane istotne zmienne
selected_features = X_train.columns[selector.support_]
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Dopasowanie modelu na wybranych zmiennych
model.fit(X_train_selected, y_train)

#odkecenie labeli
df['clarity'] = le.inverse_transform(df['clarity'])
df['cut'] = le2.inverse_transform(df['cut'])
df['color'] = le3.inverse_transform(df['color'])


# Inicjalizacja aplikacji Dash
app = dash.Dash(__name__)

# Opcje dla kontrolki wyboru kategorii
category_options = ['Rozkład zmiennych', 'Zależność ceny od innych zmiennych', 'Liczebność kategorii', 'Wizualizacja modelu regresji']

# Opcje dla kontrolki wyboru zmiennej
variable_options = ['carat', 'clarity', 'color', 'cut', 'x', 'y', 'z', 'depth', 'table']

# Układ strony
app.layout = html.Div(children=[
    html.H1(children='Diamond Data Dashboard'),

    # Kontrolka wyboru kategorii
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in category_options],
        value='Rozkład zmiennych',
        multi=False,
        style={'width': '50%'},
        clearable=False
    ),

    # Kontrolka wyboru zmiennej
    dcc.Dropdown(
        id='variable-dropdown',
        options=[{'label': variable, 'value': variable} for variable in variable_options],
        value='carat',
        multi=False,
        style={'width': '50%'},
        clearable=False
    ),

    # Wykres
    dcc.Graph(id='visualization-plot'),

    # Tabela z próbką danych
    html.Div([
        html.H3('Próbka danych'),
        dash_table.DataTable(
            id='sample-data-table',
            columns=[{'name': col, 'id': col} for col in df.columns],
            data=df.sample(10).to_dict('records')
        )
    ])
])


# Funkcja do aktualizacji wykresu i tabeli na podstawie wybranej kategorii i zmiennej
@app.callback(
    [Output('visualization-plot', 'figure'),
     Output('sample-data-table', 'data')],
    [Input('category-dropdown', 'value'),
     Input('variable-dropdown', 'value')]
)
def update_plots(selected_category, selected_variable):
    if selected_category == 'Rozkład zmiennych':
        # Aktualizacja wykresu - Rozkład zmiennej
        fig = px.histogram(df, x=selected_variable, nbins=50, title=f'Rozkład zmiennej "{selected_variable}"')
    elif selected_category == 'Zależność ceny od innych zmiennych':
        # Aktualizacja wykresu - Zależność ceny od innych zmiennych
        if selected_variable == 'carat':
            fig = px.scatter(x=X_test_selected[selected_variable], y=y_test / X_test_selected[selected_variable],
                             labels={'x': selected_variable, 'y': 'Cena / Carat'},
                             title=f'Zależność ceny od zmiennej "{selected_variable}"',
                             mode='markers', template='plotly_white')
        else:
            y_pred = model.predict(X_test_selected)
            fig = px.scatter(x=y_test, y=y_pred, labels={'x': 'Rzeczywista cena', 'y': 'Przewidziana cena'},
                             title='Zależność ceny od innych zmiennych', mode='markers', template='plotly_white')
            fig.add_trace(px.scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], mode='lines',
                                     name='Idealna zależność', template='plotly_white').data[0])
    elif selected_category == 'Liczebność kategorii':
        # Aktualizacja wykresu - Liczebność kategorii
        fig = px.bar(df[selected_variable].value_counts(), x=df[selected_variable].unique(),
                     title=f'Liczebność kategorii "{selected_variable}"', labels={'x': selected_variable, 'y': 'Liczba'},
                     template='plotly_white')
    elif selected_category == 'Wizualizacja modelu regresji':
    # Aktualizacja wykresu - Wizualizacja modelu regresji
            y_pred = model.predict(X_test_selected)
            fig = px.scatter(x=y_test, y=y_pred, labels={'x': 'Rzeczywista cena', 'y': 'Przewidziana cena'},
                     title='Wizualizacja modelu regresji', mode='markers', template='plotly_white')

    # Dodanie linii idealnej zależności
    ideal_line = go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)],
                            mode='lines', name='Idealna zależność', line={'dash': 'dash'})
    fig.add_trace(ideal_line)


    # Aktualizacja tabeli
    sample_data = df.sample(10).to_dict('records')

    return fig, sample_data


# Uruchomienie aplikacji
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: scatter() got an unexpected keyword argument 'mode'

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: scatter() got an unexpected keyword argument 'mode'

